In [1]:
import onnxruntime
import numpy as np
from PIL import Image
import glob
import os

# 1. Image preprocessing function (NHWC format)
def preprocess_image(image_path):
    image = Image.open(image_path).convert("RGB")
    image = image.resize((256, 256))  # Nudenet v2 uses 256x256 input
    img_data = np.asarray(image).astype(np.float32) / 255.0  # Normalize to [0, 1]
    img_data = np.expand_dims(img_data, axis=0)  # (1, 256, 256, 3)
    return img_data

# 2. Load the ONNX model
onnx_path = "../model/classifier_model.onnx"
session = onnxruntime.InferenceSession(onnx_path)
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name

# 3. Automatically load all image files from the folder
image_extensions = ["*.jpg", "*.jpeg", "*.png", "*.jfif", "*.bmp", "*.webp"]
image_path_list = []
for ext in image_extensions:
    image_path_list.extend(glob.glob(os.path.join("../TestImages", ext)))

# 4. Labels
labels = ['unsafe', 'safe']

# 5. Predict for each image
for image_path in image_path_list:
    input_tensor = preprocess_image(image_path)
    outputs = session.run([output_name], {input_name: input_tensor})
    preds = outputs[0][0]

    print(f"\n🖼️ File: {image_path}")
    for i, prob in enumerate(preds):
        print(f"  {labels[i]}: {prob:.4f}")
    print(f"  ==> Prediction: {labels[np.argmax(preds)]}")



🖼️ File: ../TestImages\con-cho.jpg
  unsafe: 0.0078
  safe: 0.9922
  ==> Prediction: safe

🖼️ File: ../TestImages\img_2.jpg
  unsafe: 0.9986
  safe: 0.0014
  ==> Prediction: unsafe

🖼️ File: ../TestImages\persons.jpeg
  unsafe: 0.1098
  safe: 0.8902
  ==> Prediction: safe

🖼️ File: ../TestImages\bao luc.jfif
  unsafe: 0.0175
  safe: 0.9825
  ==> Prediction: safe

🖼️ File: ../TestImages\images.jfif
  unsafe: 0.9071
  safe: 0.0929
  ==> Prediction: unsafe

🖼️ File: ../TestImages\ảnh nóng.jfif
  unsafe: 0.9954
  safe: 0.0046
  ==> Prediction: unsafe
